In [1]:
import pandas as pd
import os
import numpy as np
import pprint

In [2]:
anno = pd.read_csv("../../../sample_annotation/Data/sample_annotation.tsv", sep='\t')

In [4]:
# print(anno['FIBROBLAST_ID'])
# anno.iloc[1]
anno.loc[1,"FIBROBLAST_ID"]

'100416'

In [4]:
anno_mae = anno[anno["LAB"] == "PROKISCH"] 

In [5]:
np.unique(anno_mae["LAB"])
anno_mae.shape

(1268, 53)

In [6]:
#np.isnan(anno_mae.EXOME_ID)
anno_mae = anno_mae[pd.notnull(anno_mae.EXOME_ID)]
anno_mae = anno_mae[pd.notnull(anno_mae.RNA_ID)]

In [7]:
anno_mae.shape

(284, 53)

In [8]:
anno_mae = anno_mae[["EXOME_ID", "RNA_ID"]]

In [9]:
anno_mae['rna_file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/RNAout/paired-endout/stdFilenames/" + x + ".bam" for x in anno_mae["RNA_ID"]]

In [10]:
anno_mae['vcf_file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/exomicout/paired-endout/stdFilenames/" + x + ".vcf.gz" for x in anno_mae["EXOME_ID"]]

In [12]:
# anno_mae["vcf_file"]
anno_mae['vcf_exists'] = [os.path.exists(x) for x in anno_mae["vcf_file"]]
anno_mae['rna_exists'] = [os.path.exists(x) for x in anno_mae["rna_file"]]
anno_mae = anno_mae[anno_mae['vcf_exists'] & anno_mae['rna_exists']]

In [40]:
# ('mae/' + anno_mae["EXOME_ID"] + '-' + anno_mae["RNA_ID"] + '.Rds').tolist()

## Complete Function

In [9]:
def mae_files(anno):
    
    # subset and clean
    anno_mae = anno[anno["LAB"] == "PROKISCH"]
    anno_mae = anno_mae[pd.notnull(anno_mae.EXOME_ID)]
    anno_mae = anno_mae[pd.notnull(anno_mae.RNA_ID)]
    anno_mae = anno_mae[["EXOME_ID", "RNA_ID"]]

    # create file names
    anno_mae['rna_file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/RNAout/paired-endout/stdFilenames/" + x + ".bam" for x in anno_mae["RNA_ID"]]
    anno_mae['vcf_file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/exomicout/paired-endout/stdFilenames/" + x + ".vcf.gz" for x in anno_mae["EXOME_ID"]]

    # check for missing files
    anno_mae['vcf_exists'] = [os.path.exists(x) for x in anno_mae["vcf_file"]]
    anno_mae['rna_exists'] = [os.path.exists(x) for x in anno_mae["rna_file"]]
    anno_mae = anno_mae[anno_mae['vcf_exists'] & anno_mae['rna_exists']]
    
    vcf = anno_mae["EXOME_ID"] 
    rna = anno_mae["RNA_ID"]
    
    return vcf.tolist(), rna.tolist()

In [10]:
vcf, rna = mae_files(anno)

# Test for all vcf files

In [50]:
def all_vcf(anno):

    # subset and clean
    anno = anno[anno["LAB"] == "PROKISCH"]
    anno = anno[pd.notnull(anno.EXOME_ID)]
    anno_vcf = anno[["EXOME_ID"]].copy()

    anno_vcf['file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/exomicout/paired-endout/stdFilenames/" + x + ".vcf.gz" for x in anno_vcf["EXOME_ID"]]
    anno_vcf['vcf_exists'] = [os.path.exists(x) for x in anno_vcf["file"]]
    anno_vcf = anno_vcf[anno_vcf['vcf_exists']]
    
    return pd.unique(anno_vcf["EXOME_ID"]).tolist()

In [51]:
all_vcfs = all_vcf(anno)
len(all_vcfs)

752

In [56]:
len(vcf)

232

## OUTRIDER group files

In [3]:
anno = pd.read_csv("../../../sample_annotation/Data/sample_annotation.tsv", sep='\t')

In [4]:
anno.head()

,FIBROBLAST_ID,EXOME_ID,RNA_ID,GENOME_ID,KNOWN_MUTATION,TISSUE,DISEASE,MAX_RNA_TO_EXOME_IDENTITY,PROTEOME_ID,GENDER,...,IS_RNA_SEQ_STRANDED,BATCH,PubMed_ID,Last_revised,fib_id,PATIENT_ID,PEDIGREE_new,OUTRIDER_GROUP,CANDIDATE,remove
0,100416,NaN,MUC3196,NaN,NaN,FIBROBLAST,HEALTHY,0.41,NaN,NaN,...,1.0,Batch_2,NaN,NaN,100416,NaN,NaN,"all,fib_ss,fib,no_blood",NaN,NaN
1,100416,NaN,MUC3197,NaN,NaN,FIBROBLAST,HEALTHY,0.41,NaN,NaN,...,1.0,Batch_2,NaN,NaN,100416,NaN,NaN,"all,gal,fib,no_blood",NaN,NaN
2,100634,92582,100634R,NaN,NaN,FIBROBLAST,MITO,0.94,P100634,female,...,1.0,Batch_2,NaN,20181119.0,100634,p67077,ExDiag0510,"all,fib_ss,fib,no_blood",NaN,NaN
3,100782,98288,110468R,NaN,NaN,FIBROBLAST,MITO,NaN,P100782,male,...,1.0,Batch_4,NaN,NaN,100782,p70754,ExDiag0874,"all,fib_ss,fib,no_blood",NaN,NaN
4,101309,104905,110442R,NaN,NaN,FIBROBLAST,MITO,NaN,NaN,female,...,1.0,Batch_4,NaN,NaN,101309,p72867,SALK0009,"all,fib_ss,fib,no_blood",NaN,NaN


In [8]:
def outrider_files(anno):

    # subset and clean
    anno_outrider = anno[(anno.LAB == "PROKISCH") & pd.notnull(anno.RNA_ID) & pd.notnull(anno.OUTRIDER_GROUP)]
    anno_outrider = anno_outrider[["RNA_ID", "OUTRIDER_GROUP"]].drop_duplicates().copy()

    # create filenames and ignore missing files
    anno_outrider['file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/RNAout/"
                             for x in anno_outrider["RNA_ID"]]
    anno_outrider['file_exists'] = [os.path.exists(x) for x in anno_outrider["file"]]
    anno_outrider = anno_outrider[anno_outrider['file_exists']]

    # subset by OUTRIDER_GROUP
    outrider_groups = []
    for s in set(anno_outrider.OUTRIDER_GROUP):
        outrider_groups.extend(s.split(','))
    outrider_ids = {og : anno_outrider.loc[anno_outrider.OUTRIDER_GROUP.str.contains('(^|,)' + og + '(,|$)'), 'RNA_ID'].tolist() for og in set(outrider_groups)}
    return {og: _list for og, _list in outrider_ids.items() if len(_list) > 40}

In [9]:
of = outrider_files(anno)

/opt/modules/i12g/anaconda/3-4.1.1/envs/genetic_diagnosis/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [10]:
len(of['all'])

665

In [15]:
len(of['fib']), len(of['fib_all'])

(345, 345)

In [17]:
anno_outrider = anno[(anno.LAB == "PROKISCH") & pd.notnull(anno.RNA_ID) & pd.notnull(anno.OUTRIDER_GROUP)]
anno_outrider = anno_outrider[["RNA_ID", "OUTRIDER_GROUP"]].drop_duplicates().copy()

    # create filenames and ignore missing files
anno_outrider['file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/RNAout/"
                             for x in anno_outrider["RNA_ID"]]
anno_outrider['file_exists'] = [os.path.exists(x) for x in anno_outrider["file"]]
anno_outrider = anno_outrider[anno_outrider['file_exists']]

In [19]:
anno_outrider

,RNA_ID,OUTRIDER_GROUP,file,file_exists
0,MUC3196,"all,fib_ss,fib_all,fib,no_blood",/s/project/mitoMultiOmics/raw_data/helmholtz/M...,True
1,MUC3197,"all,gal,fib_all,no_blood",/s/project/mitoMultiOmics/raw_data/helmholtz/M...,True
2,100634R,"all,fib_ss,fib_all,fib,no_blood",/s/project/mitoMultiOmics/raw_data/helmholtz/1...,True
3,110468R,"all,fib_ss,fib_all,fib,no_blood",/s/project/mitoMultiOmics/raw_data/helmholtz/1...,True
4,110442R,"all,fib_ss,fib_all,fib,no_blood",/s/project/mitoMultiOmics/raw_data/helmholtz/1...,True
5,110460R,"all,blood",/s/project/mitoMultiOmics/raw_data/helmholtz/1...,True
6,101401R,"all,fib_ss,fib_all,fib,no_blood",/s/project/mitoMultiOmics/raw_data/helmholtz/1...,True
7,101676R,"all,fib_ss,fib_all,fib,no_blood",/s/project/mitoMultiOmics/raw_data/helmholtz/1...,True
8,102384R,"all,fib_ss,fib_all,fib,no_blood",/s/project/mitoMultiOmics/raw_data/helmholtz/1...,True
9,102385R,"all,fib_ss,fib_all,fib,no_blood",/s/project/mitoMultiOmics/raw_data/helmholtz/1...,True


In [23]:
outrider_groups = []
for s in set(anno_outrider.OUTRIDER_GROUP):
    outrider_groups.extend(s.split(','))
outrider_ids = {og : anno_outrider.loc[anno_outrider.OUTRIDER_GROUP.str.contains('(^|,)' + og + '(,|$)'), 'RNA_ID'].tolist() for og in set(outrider_groups)}
#{og: _list for og, _list in outrider_ids.items() if len(_list) > 40}

/opt/modules/i12g/anaconda/3-4.1.1/envs/genetic_diagnosis/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
